### Election Data for 2000，2004，2008

This script is used to get data from wiki for election 2000,2004,2008.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [1764]:
column_Name = ['state','county','1st', '2nd', '3rd', 'votes1','votes2', 'votes3', 'pct1', 'pct2', 'pct3', 'party1', 'party2', 'party3']

In [1765]:
# Map candidate and party map
def getCandidatePartyMap(state_soup, class_name, table_pos):
    party_content = state_soup.find_all('table',{'class':class_name})[table_pos]
    party_row = party_content.find_all('tr')
    start_flag = False
    party_cand_map = {}
    for this_row in party_row:

        if len(this_row.find_all('th'))<=1 and start_flag is False :
            continue

        if start_flag is True:
            
            this_td = this_row.find_all('td')
            if this_td[0].get_text() == '':
                this_td = this_td[1:]

            if (len(this_td) >= rec_length):
                party_index_tmp = party_index
                candidate_index_tmp = candidate_index
                
                party_tmp = this_td[party_index_tmp].get_text().strip(' \n').split()
                if not party_tmp:
                    party = None
                else:
                    party = party_tmp[0]
                
                if party == 'Republican':
                    party = 'R'
                elif party == 'Democratic' or party == 'Democrat':
                    party = 'D'
                else:
                    party = 'O'
                    
                candidate_tmp = this_td[candidate_index_tmp].get_text().strip(' \n').split(' ')
                
                if '(' in candidate_tmp[-1] or '.' in candidate_tmp[-1]:
                    candidate = candidate_tmp[-2]
                elif 'of' in candidate_tmp:
                    candidate = candidate_tmp[candidate_tmp.index("of")-1]
                else:
                    candidate = candidate_tmp[-1]
                
                party_cand_map[candidate] = party
                continue
            else:
                continue
        else:
            column_name = [c_text.get_text().strip(' \n').split()[-1].lower() for c_text in this_row.find_all('th')]

            party_index = column_name.index("party")
            candidate_index = column_name.index("candidate")
            start_flag = True
            rec_length = len(this_row.find_all('th'))

    party_cand_map['Others'] = 'O'
    party_cand_map['Other'] = 'O'
    return party_cand_map

# Map candidate and party map
def getCandidatePartyMapAlt(state_soup, class_name, table_pos):
    party_content = state_soup.find_all('table',{'class':class_name})[table_pos]
    party_row = party_content.find_all('tr')

    start_flag = False
    party_cand_map = {}
    for this_row in party_row:

        if start_flag is True:
            
            this_td = this_row.find_all('td')
            if not this_td[0].find_all('a'):
                break
            if this_td[0].find_all('a')[0].get_text() == '':
                this_td = this_td[1:]

            if (len(this_td) >= rec_length):
                party_index_tmp = party_index
                candidate_index_tmp = candidate_index
                
                party_tmp = this_td[party_index_tmp].get_text().strip(' \n').split()
                if not party_tmp:
                    party = None
                else:
                    party = party_tmp[0]
                
                if party == 'Republican':
                    party = 'R'
                elif party == 'Democratic' or party == 'Democrat':
                    party = 'D'
                else:
                    party = 'O'
                    
                candidate_tmp = this_td[candidate_index_tmp].find_all('a')[0].get_text().strip(' \n').split(' ')
                
                if '(' in candidate_tmp[-1] or '.' in candidate_tmp[-1]:
                    candidate = candidate_tmp[-2]
                elif 'of' in candidate_tmp:
                    candidate = candidate_tmp[candidate_tmp.index("of")-1]
                else:
                    candidate = candidate_tmp[-1]
                
                party_cand_map[candidate] = party
                continue
            else:
                continue
        else:
            column_name = [c_text.get_text().strip(' \n').split()[-1].lower() for c_text in this_row.find_all('th')]
            column_name[0] = 'candidate'
            party_index = column_name.index("party")
            candidate_index = column_name.index("candidate")
            start_flag = True
            rec_length = len(this_row.find_all('th'))

    party_cand_map['Others'] = 'O'
    party_cand_map['Other'] = 'O'
    return party_cand_map

# Map candidate and party map
def getCandidatePartyMapSpecify(state_soup, class_name, table_pos):
    party_cand_map['Others'] = 'O'
    party_cand_map['Other'] = 'O'
    party_cand_map['Bush'] = 'R'
    party_cand_map['Gore'] = 'D'
    return party_cand_map

# Get data for every county in the state
def getContentForState(state, state_soup,column_Name, table_info):

    if table_info[0] == 0:
        return pd.DataFrame(columns = column_Name)
    
    raw_data = pd.DataFrame(columns = column_Name) 
    county_content = state_soup.find_all('table',{'class':table_info[1]})[table_info[2]]
    county_tr = county_content.find_all('tr')
    start_col = 2 if table_info[3] == 3 or table_info[3] == 6 else 1
    start_row = 2 if table_info[3] == 4 else 1
    
    candidate = []
    
    if table_info[3] == 2 or table_info[3] == 4:
        candidate = [name.get_text().replace('#','').replace('%','').split()[-1] for name in county_tr[0].find_all('th')[start_col:start_col+3]]
    elif table_info[3] == 5:
        candidate = [name.get_text().replace('#','').replace('%','').split()[-1] for name in county_tr[0].find_all('th')[start_col:start_col+6:2]]
    elif table_info[3] == 6:
        candidate = [name.get_text().replace('#','').replace('%','').split('/')[0] for name in county_tr[0].find_all('th')[start_col:start_col+6:2]]
    elif table_info[3] == 9:
        candidate = [name.get_text().replace('#','').replace('%','').split()[0] for name in county_tr[0].find_all('td')[start_col:-1:2]]
    elif len(county_tr[0].find_all('th'))<7:
        candidate = [name.get_text().replace('#','').replace('%','').split()[0] for name in county_tr[0].find_all('th')[start_col:-1:2]]
    else:
        candidate = [name.get_text().replace('#','').replace('%','').split()[0] for name in county_tr[0].find_all('th')[start_col:start_col+6:2]]
    
    if 'Jr.' in candidate:
        candidate[candidate.index('Jr.')] = county_tr[0].find_all('th')[start_col+candidate.index('Jr.')].get_text().split()[-2]
        
    if(table_info[-1] == 0):
        party_map = getCandidatePartyMap(state_soup, table_info[4], table_info[5])
    elif(table_info[-1] == 2):
        party_map = getCandidatePartyMapSpecify(state_soup, table_info[4], table_info[5])
    else:
        party_map = getCandidatePartyMapAlt(state_soup, table_info[4], table_info[5])

    for county_stat in county_tr[start_row:]:
        if table_info[3] == 7:
            county_name = county_stat.find_all('th')[0].get_text().split(',')[0].replace(' County','')
        else:
            county_name = county_stat.find_all('td')[0].get_text().split(',')[0].replace(' County','')
        if county_name == 'Total':
            break
        county_party = [party_map[this_name] for this_name in candidate]
        
        if len(county_stat.find_all('td'))<7:
            county_num = county_stat.find_all('td')[start_col:]
        else:   
            county_num = county_stat.find_all('td')[start_col:start_col+6]

        county_num1 = county_num[0::2]
        county_num2 = county_num[1::2]
        if len(county_num1)> len(county_num2):
            county_num1 = county_num1[0:len(county_num2)]
        
        if '%' in county_num1[0].get_text() and ',' not in county_num1[0].get_text():
            county_pct = [float(k.get_text().replace('%','').replace("'",'')) for k in county_num1]
            county_vote = [int(k.get_text().replace(',','').replace("'",'').replace("?",'0').replace(" ",'').replace(".",'').replace("%",'')) for k in county_num2]
        else:
            county_pct = [float(k.get_text().replace('%','').replace("'",'')) for k in county_num2]
            county_vote = [int(k.get_text().replace(',','').replace("'",'').replace("?",'0').replace(" ",'').replace(".",'').replace("%",'')) for k in county_num1]

        for l in range(len(county_pct)):
            if county_pct[l] == 0 and county_vote[l] != 0:
                county_pct[l] = None
            elif county_pct[l] != 0 and county_vote[l] == 0:
                county_vote[l] = None
        if all(county_vote):        
            sort_index = [b[0] for b in sorted(enumerate(county_vote),key=lambda i:i[1], reverse=True)]
        else:
            sort_index = [b[0] for b in sorted(enumerate(county_pct),key=lambda i:i[1], reverse=True)]
        ordered_candidate = [candidate[i] for i in sort_index]
        county_vote = [county_vote[i] for i in sort_index]
        county_pct = [county_pct[i] for i in sort_index]
        county_party = [county_party[i] for i in sort_index]
        
        if len(ordered_candidate) < 3:
            ordered_candidate = ordered_candidate+[None]*(3-len(ordered_candidate))
            county_vote = county_vote + [None]*(3-len(county_vote))
            county_pct = county_pct + [None]*(3-len(county_pct))
            county_party = county_party + [None]*(3-len(county_party))
            
        state_name = state.replace('_',' ')
        
        this_record = [state_name, county_name] + ordered_candidate + county_vote + county_pct + county_party
        raw_data = raw_data.append(pd.DataFrame([this_record], columns= column_Name))
    return raw_data

# Get data for every county in the state
def getContentForStateShort(state, state_soup,column_Name, table_info):
    raw_data = pd.DataFrame(columns = column_Name) 
    county_content = state_soup.find_all('table',{'class':table_info[1]})[table_info[2]]
    
    county_tr = county_content.find_all('tr')
    if table_info[3] == 1:
        candidate = [name.get_text().replace('#','').replace('%','').split()[0] for name in county_tr[0].find_all('td')[1:3]]
    else:
        candidate = [name.get_text().replace('#','').replace('%','').split('(')[0] for name in county_tr[0].find_all('th')[1:3]]

    if(table_info[-1] == 0):
        party_map = getCandidatePartyMap(state_soup, table_info[4], table_info[5])
    elif(table_info[-1] == 2):
        party_map = getCandidatePartyMapSpecify(state_soup, table_info[4], table_info[5])
    else:
        party_map = getCandidatePartyMapAlt(state_soup, table_info[4], table_info[5])
        
    for county_stat in county_tr[1:]:
        county_name = county_stat.find_all('td')[0].get_text().split(',')[0].replace(' County','')
        if county_name == 'Total':
            break
        county_party = [party_map[this_name] for this_name in candidate]
        
        county_num = county_stat.find_all('td')[1:3]

        if '%' in county_num[0].get_text():
            county_pct = [float(k.get_text().replace('%','').replace("'",'')) for k in county_num]
            county_vote = [None] * len(county_pct)
            sort_index = [b[0] for b in sorted(enumerate(county_pct),key=lambda i:i[1], reverse=True)]
        else:
            county_vote = [int(k.get_text().replace(',','').replace("'",'').replace(" ",'')) for k in county_num]
            county_pct = [None] * len(county_vote)
            sort_index = [b[0] for b in sorted(enumerate(county_vote),key=lambda i:i[1], reverse=True)]

        ordered_candidate = [candidate[i] for i in sort_index]
        county_vote = [county_vote[i] for i in sort_index]
        county_pct = [county_pct[i] for i in sort_index]
        county_party = [county_party[i] for i in sort_index]
        
        if len(ordered_candidate) < 3:
            ordered_candidate = ordered_candidate+[None]*(3-len(ordered_candidate))
            county_vote = county_vote + [None]*(3-len(county_vote))
            county_pct = county_pct + [None]*(3-len(county_pct))
            county_party = county_party + [None]*(3-len(county_party))
            
        state_name = state.replace('_',' ')
        
        this_record = [state_name, county_name] + ordered_candidate + county_vote + county_pct + county_party
        raw_data = raw_data.append(pd.DataFrame([this_record], columns= column_Name))
    return raw_data


### Get Data for 2000 from Wiki

In [1809]:
url = 'https://en.wikipedia.org/wiki/United_States_presidential_election,_2000#Results_by_state'
from fake_useragent import UserAgent
ua = UserAgent()

header = {'User-Agent':str(ua.chrome)}
htmlContent = requests.get(url, headers=header)
soup = BeautifulSoup(htmlContent.text, 'html.parser')

state_name_data = soup.find_all('div', {'style':'overflow:auto'})[0].find_all('td',{'scope':'row'})

states = []
for this_state in state_name_data:
    states.append(this_state.get_text().strip(" \n★†").replace(' ','_'))


In [1810]:
# Specify the property of the party map table and county statistic table on wiki for each state
table_info_2000 = [[1, 'wikitable sortable', 0, 0, '', -1, 0]]*51
table_info_2000[0] = [1, 'wikitable sortable', 0, 3, '', -1, 0]
table_info_2000[1] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[3] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[4] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[5] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[7] = [1, 'wikitable', 0, 4, '', -1, 0]
table_info_2000[8] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[9] = [1, 'wikitable sortable', 0, 0, 'wikitable', -3, 1]
table_info_2000[10] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[11] = [1, 'wikitable', -2, 2, '', -1, 0]
table_info_2000[12] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[14] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[16] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[17] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[18] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[19] = [1, 'wikitable sortable', -1, 0, '', -1, 0]
table_info_2000[20] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[21] = [1, 'wikitable', -1, 2, '', -1, 0]
table_info_2000[22] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[23] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[24] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[25] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[26] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[27] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[30] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[31] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[32] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[33] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[34] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[35] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2000[36] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[37] = [1, 'wikitable sortable', 0, 5, '', -1, 0]
table_info_2000[38] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[39] = [1, 'wikitable', -2, 2, '', -1, 2]
table_info_2000[40] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[41] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[42] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[43] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[44] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[47] = [1, 'sortable', 0, 6, '', -1, 0]
table_info_2000[48] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[49] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2000[50] = [0, 'wikitable sortable', 0, 0, '', -1, 0]

In [1811]:
all_data_2000 = pd.DataFrame(columns = column_Name) 
for i in range(len(states)):
    this_state_url = 'https://en.wikipedia.org/wiki/United_States_presidential_election_in_'+ states[i] +',_'+'2000'
    state_htmlContent = requests.get(this_state_url, headers=header)
    state_soup = BeautifulSoup(state_htmlContent.text, 'html.parser')
    if table_info_2000[i][3] == 1:
        this_state = getContentForStateShort(states[i], state_soup,column_Name, table_info_2000[i])
    else:
        this_state = getContentForState(states[i], state_soup,column_Name, table_info_2000[i])
    all_data_2000 = all_data_2000.append(this_state)
all_data_2000 = all_data_2000.reset_index(drop = True)
all_data_2000.to_csv('2000.csv')

### Get Data for 2004 from Wiki

In [1812]:
url = 'https://en.wikipedia.org/wiki/United_States_presidential_election,_2004#Results_by_state'
from fake_useragent import UserAgent
ua = UserAgent()

header = {'User-Agent':str(ua.chrome)}
htmlContent = requests.get(url, headers=header)
soup = BeautifulSoup(htmlContent.text, 'html.parser')

state_name_data = soup.find_all('div', {'style':'overflow:auto'})[0].find_all('td',{'style':'text-align:left;'})

states = []
for this_state in state_name_data:
    states.append(this_state.get_text().strip(" \n★†").replace(' ','_'))

In [1813]:
# Specify the property of the party map table and county statistic table on wiki for each state
table_info_2004 = [[1, 'wikitable sortable', 0, 0, '', -1, 0]]*51
table_info_2004[1] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2004[8] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2004[14] = [1, 'wikitable sortable', 0, 0, 'wikitable', 0, 0]
table_info_2004[19] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2004[22] = [1, 'wikitable sortable', 0, 1, '', -1, 0]
table_info_2004[27] = [1, '', -1, 2, 'wikitable', 0, 0]
table_info_2004[34] = [1, 'wikitable sortable', 0, 0, 'wikitable', 0, 0]
table_info_2004[35] = [1, 'wikitable sortable', 0, 0, '', -2, 0]
table_info_2004[45] = [1, 'wikitable', 0, 2, '', -1, 0]

In [1814]:
all_data_2004 = pd.DataFrame(columns = column_Name) 
for i in range(len(states)):
    this_state_url = 'https://en.wikipedia.org/wiki/United_States_presidential_election_in_'+ states[i] +',_'+'2004'
    state_htmlContent = requests.get(this_state_url, headers=header)
    state_soup = BeautifulSoup(state_htmlContent.text, 'html.parser')
    if table_info_2004[i][3] == 1:
        this_state = getContentForStateShort(states[i], state_soup,column_Name, table_info_2004[i])
    else:
        this_state = getContentForState(states[i], state_soup,column_Name, table_info_2004[i])
    all_data_2004 = all_data_2004.append(this_state)
all_data_2004 = all_data_2004.reset_index(drop = True)
all_data_2004.to_csv('2004.csv')

In [1815]:
### Get Data for 2004 from Wiki

In [1816]:
url = 'https://en.wikipedia.org/wiki/United_States_presidential_election,_2004#Results_by_state'
from fake_useragent import UserAgent
ua = UserAgent()

header = {'User-Agent':str(ua.chrome)}
htmlContent = requests.get(url, headers=header)
soup = BeautifulSoup(htmlContent.text, 'html.parser')

state_name_data = soup.find_all('div', {'style':'overflow:auto'})[0].find_all('td',{'style':'text-align:left;'})

states = []
for this_state in state_name_data:
    states.append(this_state.get_text().strip(" \n★†").replace(' ','_'))

In [1817]:
table_info_2008 = [[1, 'wikitable sortable', 0, 0, '', -1, 0]]*51
table_info_2008[1] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[2] = [1, 'wikitable sortable', 0, 7, '', -1, 0]
table_info_2008[3] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[4] = [1, 'wikitable sortable', -1, 0, '', -1, 0]
table_info_2008[8] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[14] = [1, 'wikitable sortable', 0, 0, 'wikitable', 0, 0]
table_info_2008[15] = [1, 'wikitable sortable', -1, 0, '', -1, 0]
table_info_2008[17] = [1, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[19] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[20] = [1, 'wikitable sortable', 0, 8, '', -1, 0]
table_info_2008[22] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[23] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[27] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[29] = [1, 'wikitable sortable', -1, 0, '', -1, 0]
table_info_2008[30] = [1, 'wikitable sortable', -1, 9, '', -1, 0]
table_info_2008[34] = [0, 'wikitable sortable', 0, 0, '', -1, 0]
table_info_2008[47] = [0, 'wikitable sortable', 0, 0, '', -1, 0]

In [1820]:
all_data_2008 = pd.DataFrame(columns = column_Name) 
for i in range(len(states)):
    this_state_url = 'https://en.wikipedia.org/wiki/United_States_presidential_election_in_'+ states[i] +',_'+'2008'
    state_htmlContent = requests.get(this_state_url, headers=header)
    state_soup = BeautifulSoup(state_htmlContent.text, 'html.parser')
    if table_info_2008[i][3] == 1 or table_info_2008[i][3] == 8:
        this_state = getContentForStateShort(states[i], state_soup,column_Name, table_info_2008[i])
    else:
        this_state = getContentForState(states[i], state_soup,column_Name, table_info_2008[i])
    all_data_2008 = all_data_2008.append(this_state)
all_data_2008 = all_data_2008.reset_index(drop = True)
all_data_2008.to_csv('2008_wiki.csv')